In [1]:
import tensorflow as tf
import scipy.io as sio
sess = tf.InteractiveSession()



In [2]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_4x4(x):
  return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [3]:
data=sio.loadmat('D:/OSU/SongMat/104158 Aoi Eir - INNOCENCE.mat')
inputOsu=data['input1'].reshape(-1,128*128)
outputOsu=data['target1'].reshape(-1,4)

In [4]:
inputOsu.shape

(1076, 16384)

In [5]:
x = tf.placeholder("float", shape=[None, 128*128])
y_ = tf.placeholder("float", shape=[None, 4])


x_image = tf.reshape(x, [-1,128,128,1])

W_conv1 = weight_variable([4,32,1,12])
b_conv1 = bias_variable([12])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_4x4(h_conv1)


W_conv2 = weight_variable([32, 4, 12, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_4x4(h_conv2)



W_fc1 = weight_variable([8*8*36, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*36])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)




W_fc2 = weight_variable([1024, 4])
b_fc2 = bias_variable([4])



keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [ ]:
#sess.run(tf.initialize_all_variables())  


In [6]:
sess.run(tf.global_variables_initializer())


In [7]:
saver = tf.train.Saver()

step_size=100
for i in range(800):
    if i%step_size == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: inputOsu[i:i+step_size,:], y_: outputOsu[i:i+step_size,:], keep_prob: 1.0})
        print("step %d, training accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x: inputOsu[i:i+step_size,:], y_: outputOsu[i:i+step_size,:], keep_prob: 0.5})

#print "test accuracy %g"%accuracy.eval(feed_dict={
    #x: inputOsu[1200:,:,:], y_: outputOsu[1200:,:], keep_prob: 1.0})
    


step 0, training accuracy 0.23
step 100, training accuracy 0.73
step 200, training accuracy 0.77
step 300, training accuracy 0.92
step 400, training accuracy 0.89
step 500, training accuracy 0.94
step 600, training accuracy 0.91
step 700, training accuracy 0.91


In [9]:
save_path = saver.save(sess, "D:/OSU/results/innocence/save/",global_step=i+1)

In [10]:
print ("test accuracy %g"%accuracy.eval(feed_dict={
  x: inputOsu[:100,:], y_: outputOsu[:100,:], keep_prob: 1.0}))

test accuracy 0.7


In [ ]:
11*4*4

In [ ]:
12*4*4

In [ ]:
y_result=y_conv.eval(feed_dict={x: data['input'][:,:].reshape(-1,128*128), keep_prob: 1.0})

In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [11]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

[[ 22.  28.]
 [ 49.  64.]]
